In [17]:
!pip install qiskit --quiet
!pip install qiskit-machine-learning --quiet
!pip install tqdm --quiet
!pip install pylatexenc --quiet

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
from torch import Tensor, LongTensor
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import Module

from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler

from qiskit import QuantumCircuit, Aer, transpile
from qiskit_machine_learning.neural_networks import CircuitQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit.circuit import ParameterVector
from qiskit.utils import QuantumInstance

In [19]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [20]:
device = get_default_device()
device

In [21]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [22]:
# cluster state
def cluster_state_circuit(bits):
    qc = QuantumCircuit(bits)
    bits = list(range(bits))
    qc.h(bits)
    for this_bit, next_bit in zip(bits, bits[1:]):
        qc.cz(this_bit, next_bit)
    if(len(bits)!= 2):
        qc.cz(bits[0], bits[-1])
    return qc

In [23]:
circuit = cluster_state_circuit(4)
circuit.draw('mpl')

In [24]:
def one_qubit_unitary(thetas):
    qc = QuantumCircuit(1)
    qc.rx(thetas[0], 0)
    qc.ry(thetas[1], 0)
    qc.rz(thetas[2], 0)
    return qc

In [ ]:
thetas = ParameterVector('θ', length=3)
circuit = one_qubit_unitary(thetas)
circuit.draw('mpl')

In [25]:
def two_qubit_unitary(thetas):
    qc = QuantumCircuit(2)
    qc = qc.compose(one_qubit_unitary(thetas[0:3]), [0])
    qc = qc.compose(one_qubit_unitary(thetas[3:6]), [1])
    qc.rzz(thetas[6],0, 1)
    qc.ryy(thetas[7],0, 1)
    qc.rxx(thetas[8],0, 1)
    qc = qc.compose(one_qubit_unitary(thetas[9:12]), [0])
    qc = qc.compose(one_qubit_unitary(thetas[12:]), [1])
    return qc

In [ ]:
thetas = ParameterVector('θ', length=15)
circuit = two_qubit_unitary(thetas)
circuit.draw('mpl')

In [26]:
def two_qubit_pool(thetas):
    qc = QuantumCircuit(2)
    qc = qc.compose(one_qubit_unitary(thetas[0:3]), [1])
    qc = qc.compose(one_qubit_unitary(thetas[3:6]), [0])
    qc.cnot(0,1)
    qc = qc.compose(one_qubit_unitary(thetas[0:3]).inverse(), [1])
    return qc

In [ ]:
thetas = ParameterVector('θ', length=6)
circuit = two_qubit_pool(thetas)
circuit.draw('mpl')

In [27]:
def quantum_conv_circuit(bits, thetas):
    qc = QuantumCircuit(bits)
    bits = list(range(bits))
    for first, second in zip(bits[0::2], bits[1::2]):
        qc = qc.compose(two_qubit_unitary(thetas), [first,second])
    for first, second in zip(bits[1::2], bits[2::2] + [bits[0]]):
        qc = qc.compose(two_qubit_unitary(thetas), [first, second])
    return qc

In [ ]:
thetas = ParameterVector('θ', length=16)
circuit = quantum_conv_circuit(4, thetas)
circuit.draw('mpl')

In [28]:
def quantum_pool_circuit(sources, sinks, thetas):
    qc = QuantumCircuit(len(sources) + len(sinks))
    for source, sink in zip(sources, sinks):
        qc = qc.compose(two_qubit_pool(thetas), [source, sink])
    return qc

In [ ]:
thetas = ParameterVector('θ', length=6)
sources = [0,1,2,3]
sinks = [4,5,6,7]
circuit = quantum_pool_circuit(sources, sinks, thetas)
circuit.draw('mpl')

In [29]:
# Define and create QNN
def create_qcnn(n):
    backend = Aer.get_backend("aer_simulator_statevector")
    qi = QuantumInstance(backend)
    
    def last_qubit_prob(x):
        res = bin(x)[2:].zfill(2)
        return int(res[0])

    output_shape = 2  # parity = 0, 1

    in_thetas = ParameterVector('x', length=n)
    
    feature_map = QuantumCircuit(n, name="Angle Encoding")
    for i in range(n):
        feature_map.rx(in_thetas[i], i)
    
    thetas = ParameterVector('θ', length=21)
    ansatz = QuantumCircuit(n, name="QCNN")
    
    ansatz = ansatz.compose(quantum_conv_circuit(n,thetas[0:15]), range(n))
    ansatz = ansatz.compose(quantum_pool_circuit(range(n//2), range(n//2,n), thetas[15:21]))

    qc = QuantumCircuit(n)
    qc = qc.compose(transpile(cluster_state_circuit(n), backend = backend), range(n))
    qc.barrier()
    qc = qc.compose(feature_map, range(n))
    qc.barrier()
    qc = qc.compose(ansatz, range(n))

    # REMEMBER TO SET input_gradients=True FOR ENABLING HYBRID GRADIENT BACKPROP
    qcnn = CircuitQNN(
        qc,
        input_params = feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True,
        interpret=last_qubit_prob,
        output_shape=output_shape,
        quantum_instance=qi,
    )
    return qcnn

In [ ]:
qcnn = create_qcnn(2)
qcnn.circuit.draw('mpl')

In [30]:
class BlobsDataModule():
    def __init__(self, n_features = 2):
        super().__init__()
        self.batch_size = 8
        self.num_workers = 2
        
        self.dims = (n_features)
        self.output_dims = (1,)
        self.mapping = list(range(2))
        self.n_features = n_features
        
    def config(self):
        """Return important settings of the dataset, which will be passed to instantiate models."""
        return {"input_dims": self.dims, "output_dims": self.output_dims, "mapping": self.mapping}
    
    def prepare_data(self):
        self.x_train, self.y_train = make_blobs(n_samples=100,n_features=self.n_features,centers=2,random_state=42)
        self.x_val, self.y_val = make_blobs(n_samples=50,n_features=self.n_features,centers=2,random_state=42)
        self.x_test, self.y_test = make_blobs(n_samples=50,n_features=self.n_features,centers=2,random_state=42)
    
    def setup(self, stage=None):
        scaler = MinMaxScaler((-np.pi, np.pi))
        self.x_train = scaler.fit_transform(self.x_train)
        self.x_val = scaler.fit_transform(self.x_val)
        self.x_test = scaler.fit_transform(self.x_test)
        
        self.x_train = Tensor(self.x_train)
        self.y_train = Tensor(self.y_train).type(LongTensor)
        
        self.x_val = Tensor(self.x_val)
        self.y_val = Tensor(self.y_val).type(LongTensor)
        
        self.x_test = Tensor(self.x_test)
        self.y_test = Tensor(self.y_test).type(LongTensor)
        
        self.data_train = TensorDataset(self.x_train, self.y_train)
        self.data_val = TensorDataset(self.x_val, self.y_val)
        self.data_test = TensorDataset(self.x_test, self.y_test)

    def train_dataloader(self):
        return DataLoader(
            self.data_train,
            shuffle=True,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        return DataLoader(
            self.data_val,
            shuffle=False,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        return DataLoader(
            self.data_test,
            shuffle=False,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
        )
    def __repr__(self):
        basic = f"Blobs Dataset\nNum classes: {len(self.mapping)}\nMapping: {self.mapping}\nDims: {self.dims}\n"
        x, y = next(iter(self.train_dataloader()))
        data = (
            f"Train/val/test sizes: {len(self.data_train)}, {len(self.data_val)}, {len(self.data_test)}\n"
            f"Batch x stats: {(x.shape, x.dtype, x.min(), x.mean(), x.std(), x.max())}\n"
            f"Batch y stats: {(y.shape, y.dtype, y.min(), y.max())}\n"
        )
        return basic + data

In [31]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [32]:
data = BlobsDataModule()
data.prepare_data()
data.setup()

In [33]:
data

In [34]:
x_train = data.data_train.tensors[0]
y_train = data.data_train.tensors[1]

for x, y_target in zip(x_train, y_train):
    if y_target == 1:
        plt.plot(x[0], x[1], "bo")
    else:
        plt.plot(x[0], x[1], "go")
plt.plot([-3,3],[-3,3], "--", color="black")
plt.title("Train batch")
plt.show()

In [35]:
x_test = data.data_test.tensors[0]
y_test = data.data_test.tensors[1]

for x, y_target in zip(x_test, y_test):
    if y_target == 1:
        plt.plot(x[0], x[1], "bo")
    else:
        plt.plot(x[0], x[1], "go")
plt.plot([-3,3],[-3,3], "--", color="black")
plt.title("Test batch")
plt.show()

In [36]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [37]:
class QCNN(Module):
    def __init__(self,data_config):
        super().__init__()
        
        input_dims = data_config["input_dims"]
        num_classes = len(data_config["mapping"])
        
        self.qcnn = TorchConnector(create_qcnn(input_dims))
    
    def forward(self, x):
        out = self.qcnn(x)  # apply QCNN
        return out
    
    def training_step(self, batch): 
        x, y = batch
        logits = self(x)
        loss = torch.nn.NLLLoss()(logits, y)
        acc = accuracy(logits, y)
        return {'train_loss': loss, 'train_acc': acc}

    def validation_step(self, batch): 
        x, y = batch
        logits = self(x)
        loss = torch.nn.NLLLoss()(logits, y)
        acc = accuracy(logits, y)
        return {'val_loss': loss, 'val_acc': acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

In [38]:
def evaluate(model, val_dataloader):
    outputs = [model.validation_step(batch) for batch in val_dataloader]
    return model.validation_epoch_end(outputs)

In [47]:
def fit(epochs, lr, model, train_dataloader, val_dataloader,):
    history = {
        'train_acc':[],
        'train_loss':[],
        'val_acc':[np.nan],
        'val_loss':[np.nan]
    }
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=3)
    for epoch in range(epochs):
        # Training 
        with tqdm(train_dataloader, unit="batch") as tepoch:
            for batch in tepoch:
                tepoch.set_description(f"Epoch {epoch}")
                
                output = model.training_step(batch)
                loss , acc = output['train_loss'], output['train_acc']
                history['train_loss'].append(loss.item())
                history['train_acc'].append(acc.item())
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                
                tepoch.set_postfix(loss=loss.item(), 
                                   acc=acc.item(),
                                   val_loss= history['val_loss'][-1], 
                                   val_acc =history['val_acc'][-1])
                   
        # Validation
        result = evaluate(model, val_dataloader)
        val_loss = result['val_loss']
        val_acc = result['val_acc']
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        scheduler.step(val_loss)
        
    return history

In [48]:
train_dataloader = DeviceDataLoader(data.train_dataloader(), device)
val_dataloader = DeviceDataLoader(data.val_dataloader(), device)

In [49]:
model = QCNN(data.config())
to_device(model,device)

In [50]:
history = fit(10, 0.001, model, train_dataloader, val_dataloader)

In [51]:
train_losses = history['train_loss']
val_losses = history['val_loss']
train_accs = history['train_acc']
val_accs = history['val_acc']

In [52]:
plt.plot(train_losses, color='r', label='train loss')
plt.plot(train_accs, color='g', label='train accs')
plt.legend()

In [53]:
plt.plot(val_losses, color='b', label='=val loss')
plt.plot(val_accs, color='y', label='val accs')
plt.legend()

In [54]:
torch.save(model.state_dict(), "model.pt")

In [55]:
model = QCNN(data.config())
model.load_state_dict(torch.load("model.pt"))

In [56]:
test_dataloader = DeviceDataLoader(data.test_dataloader(), device)
to_device(model, device)

In [57]:
result = evaluate(model, test_dataloader)
print('test_loss: {:.4f}'.format(result['val_loss']) )
print('test_acc: {:.4f}'.format(result['val_acc']))

In [61]:
# Evaluate model and compute accuracy
y_predict = []
x_test = []
y_test = []
for data, target in test_dataloader:
    x_test += list(data.cpu().detach().numpy())
    output = model(data).cpu().detach().numpy()
    preds = np.array([0 if out[0] >= out[1] else 1 for out in output])
    y_predict = np.append(y_predict, preds)
    y_test = np.append(y_test, target.cpu().detach().numpy())
print("Accuracy:", np.sum(np.equal(y_predict , y_test)) / len(y_test))

# Plot results
# red == wrongly classified
for x, y_target, y_p in zip(x_test, y_test, y_predict):
    if y_target == 1.0:
        plt.plot(x[0], x[1], "bo")
    else:
        plt.plot(x[0], x[1], "go")
    if y_target != y_p:
        plt.scatter(x[0], x[1], s=200, facecolors="none", edgecolors="r", linewidths=2)
plt.plot([-3,3],[-3,3], "--", color="black")
plt.title("Predictions")
plt.show()